In [1]:
"""This code calculated to use a high res. topo tiff, setup matfile output and still water levels + SLR
to get flood depths in a tiff. Setup matfile output is interpolated on to the topo tiff to get total flood depths.
Written by Taran Kalra, July 22, 2024"""


'This code calculated to use a high res. topo tiff, setup matfile output and still water levels + SLR\nto get flood depths in a tiff. Setup matfile output is interpolated on to the topo tiff to get total flood depths.\nWritten by Taran Kalra, July 22, 2024'

In [5]:
from platform import python_version

print(python_version())

3.11.5


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio as rio
from create_raster import make_swan_raster_StructuredGrid

In [3]:
cases = ["1perc"]#, "2perc", "5perc", "10perc", "20perc", "50perc"]
TWL = [    1.5]#,      1.5,    1.5,     1.5,       1.5,  1.5 ]
scenario = "Restored"
scenario_out = "no_alt4"

outfile  = 'test.tif'

# input file names
gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\restored_USVI_rasterio.tif'
input_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs'
output_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Output_proc\Flood_depth\1perc_no_alt5.tif'

In [4]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd

input_file = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Output_proc\Flood_depth\1perc_no_alt5.tif'# Open the raster TIFF file
with rasterio.open(input_file) as src:
    # Read the raster data
    image = src.read(1)
    # Get the raster transform
    transform = src.transform

# Get the shapes from the raster data
results = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) 
    in enumerate(
        shapes(image, mask=None, transform=transform)
    )
)

# Create a GeoDataFrame from the shapes
gdf = gpd.GeoDataFrame.from_features(list(results))

# Save the GeoDataFrame to a shapefile
gdf.to_file('output.shp')

KeyboardInterrupt: 

In [ ]:
# Loop over the filenames in the directory
for i, case in enumerate(cases):
    matfile = f'{input_directory}\{scenario}_Reef\{case}\{case}_restored_outs.mat'
    outfile = f'{output_directory}\{case}_{scenario_out}.tif' 

    print(matfile)
    print(outfile) 
    
# Temporary filenames   
    outfile_coarse_raster = 'output_raster_coarse_Hsig4.tif'
    outfile_refine_raster = 'output_raster_refine_Hsig4.tif'    
    
    # Get the Hsig 
    make_swan_raster_StructuredGrid(outfile_coarse_raster, matfile, varname, EPSG, xMin, yMin, gridx, gridy, gridsize, angle_degrees)

    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(gridfile) 
    topobathy = finer_xds.squeeze()
    # Curate topbthy 
    topobathy = np.where(topobathy<-9999, np.nan, topobathy)

    # Open this newly created tiff on the SWAN grid 
    coarse_data = rioxarray.open_rasterio(outfile_coarse_raster)    
    # Interpolate wave setup from SWAN grid to the topobathy tiff (merged tiff)
    ds_new = coarse_data.interp(y=finer_xds.y, x=finer_xds.x, method='nearest')
    ds_new = ds_new.squeeze() #This is now wave setup interpolated over the SWAN grid

    # Fill the array to get flood depth, note TWL is variable for all scenarios
    #print(TWL[i])
    #ds_new = ds_new #+ TWL[i] - topobathy # topobathy is in meter from NOAA has reverse sign (ocean is negative, land is positive) 
    ds_new = topobathy 
    print("Converting meters to feet")
    ds_new = m_to_feet*ds_new # (ds_new is in feet)
    
    ds_new.rio.to_raster(outfile_refine_raster) # save the new array in a raster before changing the profile of the raster 

    # Grid file used to copy the profile 
    with rasterio.open(gridfile, 'r') as src:
        profile1 = src.profile.copy()

    #   the profile in the output raster1 and then add feet in units 
    with rasterio.open(outfile_refine_raster) as src: 
        profile = src.profile
        profile = profile1 
        array = src.read(1)
    # Update the profile to change the units from meters to feet    
    profile['units'] = 'Feet'

    # Save the updated array to a new TIFF file
    with rasterio.open(outfile, 'w+', **profile) as dst:
        dst.write(ds_new, 1)
        dst.update_tags(units=profile['units'])

    del outfile_refine_raster, outfile_coarse_raster

In [ ]:
print(np.nanmin(ds_new))
print(np.nanmax(ds_new))

In [ ]:

#fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(1,1,1)
#z1_plot = ax.pcolormesh(fine_data.x.values, fine_data.y.values, array, cmap='viridis', vmin=-20, vmax=20)
#plt.colorbar(z1_plot)

In [ ]:
res = finer_xds.rio.resolution()
print(res)
fine_grid = finer_xds.squeeze()
print(np.shape(fine_grid))
finer_xds
